In [1]:
import pandas as pd
from janome.tokenizer import Tokenizer
import re
import unicodedata
from tqdm import tqdm

In [2]:
train_raw_df = pd.read_csv("../../data/train.txt", sep="\t")
val_raw_df = pd.read_csv("../../data/validation.txt", sep="\t")

In [3]:
train_raw_df

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Disgust,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1,2012/07/31 23:48,train,0,1,2,1,1,0,...,1,0,0,2,0,0,0,0,0,0
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,1,2012/08/02 23:09,train,3,0,3,0,0,0,...,0,1,1,0,0,2,0,0,0,0
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1,2012/08/05 00:50,train,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,眠い、眠れない。,1,2012/08/08 01:36,train,0,2,1,0,0,1,...,2,0,0,1,0,0,0,0,1,0
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,1,2012/08/09 22:24,train,2,1,3,2,0,1,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,ジョージア移住系ブロガーになるって言っても、月間ボリューム100～1000かいな！,76,2019/10/26 16:55,train,0,2,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
39996,イギリスは6ヶ月未満、ビザなし滞在OK。凄い！,76,2019/10/26 23:24,train,0,0,0,3,0,0,...,0,0,1,0,0,2,0,0,0,0
39997,カナダは、6ヶ月以内の観光の場合、ビザなし滞在OK。凄い！,76,2019/10/26 23:26,train,0,0,0,3,0,0,...,0,0,1,0,0,2,0,0,0,0
39998,なぜ有料noteが重宝されているのかを解説したいが、需要がないので止めておこう。,76,2019/10/27 09:10,train,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [4]:
val_raw_df

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Disgust,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust
0,福岡に移住して、特に不満はない。\nけれど、慣れて来たので、他の街にも行ってみたくなってきて...,76,2019/10/27 10:46,train,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,うまくいった時の慢心には注意Σ(・∀・；),76,2019/10/27 10:52,train,0,0,0,0,0,2,...,0,0,0,0,0,0,0,1,0,0
2,お金持ちではなかったので、自分でカードゲームを作りだしたりして遊んでいた。\n\nその時の楽...,76,2019/10/27 11:23,train,0,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
3,飲み屋でしか聞けなかったような話って実はYouTubeで需要がある。,76,2019/10/27 15:11,train,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,①まずやってみる\n②疑問が出てきてから、その疑問解決のための教材を買う\n\n↑教材が高い...,76,2019/10/27 20:12,train,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,真夜中にふと思い立ち、ノートPCを持って部屋を出て、ダイニングで仕事したらすんごい捗った。\...,80,2020/09/15 08:01,train,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3196,ぐっどこんでぃしょん。\n心も頭もクリア。\n秋分の日のおかげかな？\n人と自然としっとり過...,80,2020/09/22 01:52,train,1,0,1,0,0,0,...,0,0,2,0,2,0,0,0,0,0
3197,朝から免許の更新へ。\n90分で終わり、出口へ向かうと献血の呼びかけが。\nみんな通り過ぎて...,80,2020/09/23 22:32,train,2,0,2,1,0,0,...,0,0,2,0,0,0,0,0,0,0
3198,夜も更けて参りましたが、食後のコーヒーが飲みたいのでドリップ開始…\n\nぼんやり秋の夜長を...,80,2020/10/11 00:12,train,2,0,1,0,0,0,...,0,0,0,0,2,0,0,0,0,0


In [5]:
tokenizer = Tokenizer()

In [6]:
text = "長野大学きたけど駐車場が結構いっぱいだなー( ?????;)\\n構内通り抜けて一番上の砂利のとこはまだ停めれる??\\(´ω` )/????\\(\u3000´)/????\\( ´ω`)/??"
unicodedata.normalize("NFKC", text)

'長野大学きたけど駐車場が結構いっぱいだなー( ?????;)\\n構内通り抜けて一番上の砂利のとこはまだ停めれる??\\( ́ω` )/????\\(  ́)/????\\(  ́ω`)/??'

In [7]:
for token in tokenizer.tokenize(text):
    print(token)

長野大学	名詞,固有名詞,組織,*,*,*,長野大学,ナガノダイガク,ナガノダイガク
き	動詞,自立,*,*,カ変・クル,連用形,くる,キ,キ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
けど	助詞,接続助詞,*,*,*,*,けど,ケド,ケド
駐車	名詞,サ変接続,*,*,*,*,駐車,チュウシャ,チューシャ
場	名詞,接尾,一般,*,*,*,場,ジョウ,ジョー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
結構	名詞,形容動詞語幹,*,*,*,*,結構,ケッコウ,ケッコー
いっぱい	名詞,接尾,副詞可能,*,*,*,いっぱい,イッパイ,イッパイ
だ	助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ
なー	助詞,終助詞,*,*,*,*,なー,ナー,ナー
(	名詞,サ変接続,*,*,*,*,(,*,*
 	記号,空白,*,*,*,*, ,*,*
?????;)\	名詞,サ変接続,*,*,*,*,?????;)\,*,*
n	名詞,一般,*,*,*,*,n,*,*
構内	名詞,一般,*,*,*,*,構内,コウナイ,コーナイ
通り抜け	動詞,自立,*,*,一段,連用形,通り抜ける,トオリヌケ,トーリヌケ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
一番	名詞,副詞可能,*,*,*,*,一番,イチバン,イチバン
上	名詞,接尾,副詞可能,*,*,*,上,ジョウ,ジョー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
砂利	名詞,一般,*,*,*,*,砂利,ジャリ,ジャリ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
とこ	名詞,一般,*,*,*,*,とこ,トコ,トコ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
停め	動詞,自立,*,*,一段,未然形,停める,トメ,トメ
れる	動詞,接尾,*,*,一段,基本形,れる,レル,レル
??\(´	名詞,サ変接続,*,*,*,*,??\(´,*,*
ω	記号,アルファベット,*,*,*,*,ω,オメガ,オメガ
`	名詞,サ変接続,*,*,*,*,`,*,*
 	記号,空白,*,*,*,*, ,*,*
)/????\(　´)/????\(	名詞,サ変接続,*,*,*,*,)/????\(　´)/????\(,*,*
 	記号,空白,*,*,*,*, ,*,*
´	

In [8]:
unicodedata.normalize("NFKC", "～ー")

'~ー'

In [30]:
PAT_SYM = re.compile(r"(\s|(\\n)|[!\?\.,。、\(\);:\\\/`\^<>\[\]{}\|\"'\*\+\-@#\$%&♪∀д ゚∇_α-ω\u0400-\u04ff])+")
PAT_SPACE = re.compile(r"\s+")
PAT_NUM = re.compile(r"\d+")
PAT_DEDUP = [(s, re.compile(f"({s})+")) for s in ["~", "ー", "w"]]
def preprocess_text(text, tokenizer):
    text = unicodedata.normalize("NFKC", text)
    text = PAT_SYM.sub(" ", text)
    text = PAT_NUM.sub("0", text)
    text = text.lower()
    
    token_filtered = []
    for token in tokenizer.tokenize(text):
        pos_tags = token.part_of_speech.split(",")
        if pos_tags[0] != "記号" or pos_tags[1] == "空白":
            token_filtered.append(token.surface)
        else:
            token_filtered.append(" ")
    text = "".join(token_filtered)
    for s, pat in PAT_DEDUP:
        text = pat.sub(s, text)
    text = PAT_SPACE.sub(" ", text).strip()
    return text

preprocess_text(text, tokenizer)

'何気なく入ったdvdコーナーで流れていたone pieceに目を奪われる これちょうどあかんとこやん 医者を 医者を教えでぐたざい゙'

In [31]:
for token in tokenizer.tokenize(preprocess_text(text, tokenizer)):
    print(token)

何気なく	形容詞,自立,*,*,形容詞・アウオ段,連用テ接続,何気ない,ナニゲナク,ナニゲナク
入っ	動詞,自立,*,*,五段・ラ行,連用タ接続,入る,ハイッ,ハイッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
dvd	名詞,一般,*,*,*,*,dvd,*,*
コーナー	名詞,一般,*,*,*,*,コーナー,コーナー,コーナー
で	助詞,格助詞,一般,*,*,*,で,デ,デ
流れ	動詞,自立,*,*,一段,連用形,流れる,ナガレ,ナガレ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
い	動詞,非自立,*,*,一段,連用形,いる,イ,イ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
one	名詞,一般,*,*,*,*,one,*,*
 	記号,空白,*,*,*,*, ,*,*
piece	名詞,一般,*,*,*,*,piece,*,*
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
目	名詞,一般,*,*,*,*,目,メ,メ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
奪わ	動詞,自立,*,*,五段・ワ行促音便,未然形,奪う,ウバワ,ウバワ
れる	動詞,接尾,*,*,一段,基本形,れる,レル,レル
 	記号,空白,*,*,*,*, ,*,*
これ	名詞,代名詞,一般,*,*,*,これ,コレ,コレ
ちょうど	副詞,一般,*,*,*,*,ちょうど,チョウド,チョード
あかん	感動詞,*,*,*,*,*,あかん,アカン,アカン
とこ	動詞,自立,*,*,五段・カ行イ音便,未然ウ接続,とく,トコ,トコ
やん	助動詞,*,*,*,不変化型,基本形,やん,ヤン,ヤン
 	記号,空白,*,*,*,*, ,*,*
医者	名詞,一般,*,*,*,*,医者,イシャ,イシャ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
 	記号,空白,*,*,*,*, ,*,*
医者	名詞,一般,*,*,*,*,医者,イシャ,イシャ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
教え	名詞,一般,*,*,*,*,教え,オシエ,オシエ
で	助詞,格助詞,一般,*,*,*,で,デ,デ
ぐたざい゙	名詞,一般,*,*,*,*,ぐたざい゙,*,*


In [32]:
train_X = pd.Series([preprocess_text(text, tokenizer) for text in tqdm(train_raw_df["Sentence"])])
val_X = pd.Series([preprocess_text(text, tokenizer) for text in tqdm(val_raw_df["Sentence"])])

100%|██████████| 3200/3200 [00:03<00:00, 876.36it/s] 


In [33]:
train_X

0                            ぼけっとしてたらこんな時間 チャリあるから食べにでたいのに
1        今日の月も白くて明るい 昨日より雲が少なくてキレイな と立ち止まる帰り道 チャリなし生活も悪くない
2                        早寝するつもりが飲み物がなくなりコンビニへ ん 今日 風が涼しいな
3                                                  眠い 眠れない
4             ただいま って新体操してるやん 外食する気満々で家に何もないのに テレビから離れられない
                               ...                        
39995                  ジョージア移住系ブロガーになるって言っても 月間ボリューム0~0かいな
39996                               イギリスは0ヶ月未満 ビザなし滞在ok 凄い
39997                         カナダは 0ヶ月以内の観光の場合 ビザなし滞在ok 凄い
39998              なぜ有料noteが重宝されているのかを解説したいが 需要がないので止めておこう
39999     ブログの良い所 興味ない所は読み飛ばせる たぶんyoutubeもそういう機能が搭載されていくはず
Length: 40000, dtype: object

In [34]:
val_X

0       福岡に移住して 特に不満はない けれど 慣れて来たので 他の街にも行ってみたくなってきている...
1                                          うまくいった時の慢心には注意
2       お金持ちではなかったので 自分でカードゲームを作りだしたりして遊んでいた その時の楽しい経験...
3                       飲み屋でしか聞けなかったような話って実はyoutubeで需要がある
4       0まずやってみる 0疑問が出てきてから その疑問解決のための教材を買う 教材が高い場合は 先...
                              ...                        
3195    真夜中にふと思い立ち ノートpcを持って部屋を出て ダイニングで仕事したらすんごい捗った 環...
3196       ぐっどこんでぃしょん 心も頭もクリア 秋分の日のおかげかな 人と自然としっとり過ごしたい気分
3197    朝から免許の更新へ 0分で終わり 出口へ向かうと献血の呼びかけが みんな通り過ぎていく 献血...
3198    夜も更けて参りましたが 食後のコーヒーが飲みたいのでドリップ開始 ぼんやり秋の夜長を楽しみた...
3199      コーヒー休憩 kahavitauko いつもの豆なのにすごく美味しくできた レシピは記録したv
Length: 3200, dtype: object

In [35]:
train_X.to_csv("../../data/v2/train_normalized.txt", index=False, header=False, sep="\t")
val_X.to_csv("../../data/v2/validation_normalized.txt", index=False, header=False, sep="\t")

In [15]:
text = "何気なく入ったdvdコーナーで流れていたone pieceに目を奪われる これちょうどあかんとこやん 医者を 医者を教えでぐたざい゙"
for token in tokenizer.tokenize(preprocess_text(text, tokenizer)):
    print(token)

何気なく	形容詞,自立,*,*,形容詞・アウオ段,連用テ接続,何気ない,ナニゲナク,ナニゲナク
入っ	動詞,自立,*,*,五段・ラ行,連用タ接続,入る,ハイッ,ハイッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
dvd	名詞,一般,*,*,*,*,dvd,*,*
コーナー	名詞,一般,*,*,*,*,コーナー,コーナー,コーナー
で	助詞,格助詞,一般,*,*,*,で,デ,デ
流れ	動詞,自立,*,*,一段,連用形,流れる,ナガレ,ナガレ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
い	動詞,非自立,*,*,一段,連用形,いる,イ,イ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
one	名詞,一般,*,*,*,*,one,*,*
 	記号,空白,*,*,*,*, ,*,*
piece	名詞,一般,*,*,*,*,piece,*,*
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
目	名詞,一般,*,*,*,*,目,メ,メ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
奪わ	動詞,自立,*,*,五段・ワ行促音便,未然形,奪う,ウバワ,ウバワ
れる	動詞,接尾,*,*,一段,基本形,れる,レル,レル
 	記号,空白,*,*,*,*, ,*,*
これ	名詞,代名詞,一般,*,*,*,これ,コレ,コレ
ちょうど	副詞,一般,*,*,*,*,ちょうど,チョウド,チョード
あかん	感動詞,*,*,*,*,*,あかん,アカン,アカン
とこ	動詞,自立,*,*,五段・カ行イ音便,未然ウ接続,とく,トコ,トコ
やん	助動詞,*,*,*,不変化型,基本形,やん,ヤン,ヤン
 	記号,空白,*,*,*,*, ,*,*
医者	名詞,一般,*,*,*,*,医者,イシャ,イシャ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
 	記号,空白,*,*,*,*, ,*,*
医者	名詞,一般,*,*,*,*,医者,イシャ,イシャ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
教え	名詞,一般,*,*,*,*,教え,オシエ,オシエ
で	助詞,格助詞,一般,*,*,*,で,デ,デ
ぐたざい゙	名詞,一般,*,*,*,*,ぐたざい゙,*,*
